# Martial Arts StackExchange Analysis

### Author: Aleksandra Kłos

For this project, our group decided to analyse Martial Arts datasets from https://archive.org/download/stackexchange.


The structure of each dataset is as follows:
* **Badges** - contain information about the badges awarded to users.
* **Comments** - include details about the comments made on post.
* **Post History** -  records changes to posts, such as edits.
* **Post Links** - information about links between posts.
* **Posts** -  data about questions and answers posted.
* **Tags** -  details the tags used for posts.
* **Users** - contain information about the users's platform. 
* **Votes** -  information about the votes cast on posts.

After downloading data from the above website and exctacting them from .zip file, I observed that they are saved in XML format. In order to be able to use them in consecutive steps, I load them into the data frames.

In [15]:
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


def load_xml_to_dataframe(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
    all_records = []

    for child in root:
        record = {}
        for key, value in child.attrib.items():
            record[key] = value
        all_records.append(record)

    return pd.DataFrame(all_records)


# File paths for the newly provided XML files
tags_file_path = 'C:\\Users\\aleks\\OneDrive\\Pulpit\\HW3\\Tags.xml'
users_file_path = 'C:\\Users\\aleks\\OneDrive\\Pulpit\\HW3\\Users.xml'
votes_file_path = 'C:\\Users\\aleks\\OneDrive\\Pulpit\\HW3\\Votes.xml'
comments_file_path = 'C:\\Users\\aleks\\OneDrive\\Pulpit\\HW3\\Comments.xml'
badges_file_path = 'C:\\Users\\aleks\\OneDrive\\Pulpit\\HW3\\Badges.xml'
postHistory_file_path = 'C:\\Users\\aleks\\OneDrive\\Pulpit\\HW3\\PostHistory.xml'
postLinks_file_path = 'C:\\Users\\aleks\\OneDrive\\Pulpit\\HW3\\PostLinks.xml'
posts_file_path = 'C:\\Users\\aleks\\OneDrive\\Pulpit\\HW3\\Posts.xml'

# Load the XML files into DataFrames
df_tags = load_xml_to_dataframe(tags_file_path)
df_users = load_xml_to_dataframe(users_file_path)
df_votes = load_xml_to_dataframe(votes_file_path)
df_comments = load_xml_to_dataframe(comments_file_path)
df_badges = load_xml_to_dataframe(badges_file_path)
df_postHistory = load_xml_to_dataframe(postHistory_file_path)
df_postLinks = load_xml_to_dataframe(postLinks_file_path)
df_posts = load_xml_to_dataframe(posts_file_path)

# Convert 'CreationDate' in Users dataframe to datetime for analysis
df_users['CreationDate'] = pd.to_datetime(df_users['CreationDate'])

# Resample to count number of users registered each year
user_registration_trends = df_users['CreationDate'].dt.to_period("Y").value_counts().sort_index()

print(user_registration_trends)


CreationDate
2012     814
2013     613
2014     797
2015     966
2016    1061
2017    1024
2018     797
2019     721
2020     702
2021     862
2022     662
2023     695
Freq: A-DEC, Name: count, dtype: int64


## Basic Research Questions

For Bicycles, Martial Arts datasets I created 4 basic research questions for which I have sought for answers. These are as follows:

### Research Question 1

#### *How have number of Posts, Comments and Votes changed over time?* 


In [19]:
# 1. Plot the number of posts, comments and votes over time to analyze how user engagement has evolved.

import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Convert 'CreationDate' in Posts, Comments, and Votes dataframes to datetime for analysis
df_posts['CreationDate'] = pd.to_datetime(df_posts['CreationDate'])
df_comments['CreationDate'] = pd.to_datetime(df_comments['CreationDate'])
df_votes['CreationDate'] = pd.to_datetime(df_votes['CreationDate'])

# Resample to count number of Posts, Comments, and Votes each year
df_posts['Year'] = df_posts['CreationDate'].dt.to_period("Y")
df_comments['Year'] = df_comments['CreationDate'].dt.to_period("Y")
df_votes['Year'] = df_votes['CreationDate'].dt.to_period("Y")

posts_per_year = df_posts.groupby('Year').size().sort_index()
comments_per_year = df_comments.groupby('Year').size().sort_index()
votes_per_year = df_votes.groupby('Year').size().sort_index()

# Convert the period index to string for efficient plotting
posts_per_year.index = posts_per_year.index.astype(str)
comments_per_year.index = comments_per_year.index.astype(str)
votes_per_year.index = votes_per_year.index.astype(str)

# Create subplot
fig = make_subplots()

# Add traces for posts, comments, and votes
fig.add_trace(go.Scatter(x=posts_per_year.index, y=posts_per_year, mode='lines+markers', name='Posts'))
fig.add_trace(go.Scatter(x=comments_per_year.index, y=comments_per_year, mode='lines+markers', name='Comments'))
fig.add_trace(go.Scatter(x=votes_per_year.index, y=votes_per_year, mode='lines+markers', name='Votes'))

# Update the layout
fig.update_layout(title='User Activity Trends Over Time',
                  xaxis_title='Year',
                  yaxis_title='Count',
                  legend_title='Activity Type',
                  xaxis=dict(showline=True, showgrid=False, showticklabels=True, linecolor='rgb(204, 204, 204)'),
                  yaxis=dict(showline=True, showgrid=False, showticklabels=True),
                  autosize=False,
                  margin=dict(autoexpand=False, l=100, r=20, t=110),
                  plot_bgcolor='white')

fig.show()

The graph titled *User Activity Trends Over Time* displays the count of different types of user activities on a Stack Exchange site from 2012 to 2023. The activities are categorized into three types: Votes (green line), Comments (orange line), and Posts (purple line). 

The observed trends for each activity are as follows:

*Votes* 
- There was a sharp decline in voting activity around 2013.
- From 2014 to the beginning of 2015 there was a slight increase in a voting trend.
- From 2015 to the beginning  of 2019, the trend in voting activity was relatively stable, yet with a slight decline.
- Starting around 2019, there is a visible downward trend, with a significant drop in voting activity over the next few years up to 2023.


*Comments*
- Comments activity started relatively low since 2012.
- There was an increase in commenting activity from 2013 to 2016, though not as significant increase as the spikes in Votes.
- After 2016, the trend in Comments declines slightly but remains more stable compared to Votes.
- From 2019 to 2020 there was a visible increase in number of Comments.
- From 2020 onward, commenting activity seems to experience a gradual and steady decrease, similar to the trend observed in voting.


*Posts*
- Despite vivid decline, posting activity begins at the lowest count compared to Votes and Comments in 2012.
- There is a steady increase in Posts from 2013, peaking in 2016 with 1057 Posts.
- Unlike Votes, after the peak in 2016, the post count does not drop sharply but rather gradually decreases over the years.
- The trend in Posts remains relatively flat compared to votes and comments from about 2019 to 2021.
- There is a slight rise in activity around 2021, which could indicate a renewed interest or a community change due to COVID-19.
- However, similar to the other activities, Posts also show a decline from 2020 onwards, continuing to decrease into 2023.

### Research Question 2

### *What is average time taken to receive an answer for a question?*

In [20]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px

# Convert 'CreationDate' to datetime
df_posts['CreationDate'] = pd.to_datetime(df_posts['CreationDate'])

# Filter questions and answers
questions = df_posts[df_posts['PostTypeId'] == '1']
answers = df_posts[df_posts['PostTypeId'] == '2']

# Merge questions with their accepted answers
merged_df = questions.merge(answers, left_on='AcceptedAnswerId', right_on='Id', suffixes=('_ques', '_ans'))

# Calculate time difference in hours or days 
merged_df['time_to_answer'] = (merged_df['CreationDate_ans'] - merged_df['CreationDate_ques']).dt.total_seconds() / 3600  # time in hours

# Calculate the average time to answer
average_time_to_answer = merged_df['time_to_answer'].mean()

print(average_time_to_answer)

# Plot the distribution of response times
fig = px.histogram(merged_df, x='time_to_answer',
                   nbins=100, log_y=True, 
                   title='Distribution of Time to Answer (in Hours)',
                   labels={'time_to_answer': 'Time to Answer (Hours)'},
                   color_discrete_sequence=['blue'])

# Update layout for better readability
fig.update_layout(xaxis=dict(title='Time to Answer (Hours)'),
                  yaxis=dict(title='Frequency'),
                  bargap=0.1)

fig.show()

# Remove outliers and recalculate the average time to answer
threshold_value = 5000  # Chosen treshold
filtered_df = merged_df[merged_df['time_to_answer'] <= threshold_value]
new_average_time_to_answer = filtered_df['time_to_answer'].mean()

print(f"New average time to answer (without outliers): {new_average_time_to_answer:.2f} hours")

522.1719145725228


New average time to answer (without outliers): 124.63 hours


The provided histogram depicts the frequency of answers over various time intervals measured in hours. The visible trends are as follows:
* There's a significant concentration of bars at the beginning of the histogram, which suggests that a large number of questions receive answers relatively quickly, within the first few hours after being posted.

* The histogram shows what appears to be a long-tail distribution. This means that while most questions are answered quickly, there is a tail of questions that take much longer to receive an answer. These could be more complex questions, less popular topics, or questions that require very specific expertise. 

### Research Question 3

### *What type of Posts (questions/answers) receive the most upvotes?*

In [21]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px

# Convert Score to numeric
df_posts['Score'] = pd.to_numeric(df_posts['Score'], errors='coerce')

# Filter questions and answers
questions = df_posts[df_posts['PostTypeId'] == '1']
answers = df_posts[df_posts['PostTypeId'] == '2']

# Calculate average score for questions and answers
average_score_questions = questions['Score'].mean()
average_score_answers = answers['Score'].mean()

# Create data for the pie chart
data = {
    'Type': ['Questions', 'Answers'],
    'Average Score': [average_score_questions, average_score_answers]
}

# Create a dataframe
df_avg_score = pd.DataFrame(data)

# Create an interactive pie chart
fig = px.pie(df_avg_score, names='Type', values='Average Score',
             title='Average Upvotes for Questions vs Answers',
             color_discrete_sequence=['#ff9999','#66b3ff'])

fig.show()

The above piechart shows the proportion of average upvotes received by Questions and Answers. Based on it, the trends are as follows:
* The **blue segment**, representing Answers, accounts for 38.1% of the average upvotes. This is a visibly smaller portion compared to the red segment, representing Questions.

* The **red segment**, representing Questions, accounts for 61.9% of the average upvotes. This suggests that on Average, Questions receive more upvotes than Answers do.

* The higher percentage of upvotes for Questions might indicate that the community values solutions seeking more than the act of answering them.

### Research Question 4

### *Is there a correlation between earning certain badges and increased User Activity?*

In [9]:
import pandas as pd
import plotly.graph_objs as go
from scipy.stats import linregress

# Convert UserId in df_badges and OwnerUserId in df_posts to numeric for proper merging
df_badges['UserId'] = pd.to_numeric(df_badges['UserId'], errors='coerce')
df_posts['OwnerUserId'] = pd.to_numeric(df_posts['OwnerUserId'], errors='coerce')

# Count badges per user
badge_counts = df_badges.groupby('UserId').size().reset_index(name='BadgeCount')

# Count posts per user
post_counts = df_posts.groupby('OwnerUserId').size().reset_index(name='PostCount')

# Merge badge count with post count
df_user_activity = badge_counts.merge(post_counts, left_on='UserId', right_on='OwnerUserId', how='inner')

# Calculate correlation
activity_correlation = df_user_activity['BadgeCount'].corr(df_user_activity['PostCount'])
print(f"Correlation between number of badges earned and user activity (posts): {activity_correlation:.3f}")

# Calculate regression line
slope, intercept, r_value, p_value, std_err = linregress(df_user_activity['BadgeCount'], df_user_activity['PostCount'])
df_user_activity['RegressionLine'] = slope * df_user_activity['BadgeCount'] + intercept

# Create interactive scatter plot with regression line
fig = go.Figure()

# Add scatter plot
fig.add_trace(go.Scatter(x=df_user_activity['BadgeCount'], y=df_user_activity['PostCount'], 
                         mode='markers', name='Data Points', marker=dict(color='blue', opacity=0.5)))

# Add regression line
fig.add_trace(go.Scatter(x=df_user_activity['BadgeCount'], y=df_user_activity['RegressionLine'], 
                         mode='lines', name='Regression Line', line=dict(color='red')))

# Update layout
fig.update_layout(title='Correlation between Number of Badges Earned and User Activity',
                  xaxis_title='Number of Badges',
                  yaxis_title='Number of Posts',
                  showlegend=True)

fig.show()

print(f"Regression Line: y = {slope:.2f}x + {intercept:.2f}")
print(f"R-squared value: {r_value**2:.3f}")

Correlation between number of badges earned and user activity (posts): 0.829


Regression Line: y = 1.73x + -4.70
R-squared value: 0.688


Each blue dot represents an individual User, with their number of badges on the x-axis and their Post count on the y-axis. Most data points cluster towards the lower left of the graph, indicating that many users have fewer Badges and Posts. The red line however is a regression line. It represents the best fit through the data points, showing the average trend in the dataset. 

The slope of the regression line (1.73) indicates that for each additional badge earned, there is an average increase of about 1.73 posts. The negative y-intercept (-4.70) suggests that a user with zero badges would be expected to have made slightly less than five posts, according to the model. However, since users cannot have negative posts, the y-intercept here is more a feature of the mathematical model rather than a practically interpretable quantity. Additionally, the correlation coeffcient value of 0.829 indicates there is strong positive corellation between the data. 

Moreover, the R-squared value, indicating how well the data fit the regression model on a scale from 0 to 1, with 1 being a perfect fit, suggests that approximately 68.8% of the variability in the number of posts can be explained by the number of badges earned. This is a strong relationship, indicating that badges are a good indicator of user activity on the forum. 

## Advanced Research Questions

### Research Question 1

### *What is the general sentiment of Comments in different topics?*

In [39]:
import pandas as pd
from lxml import etree
from textblob import TextBlob
import re

# Function to clean text data
def clean_text(text):
    # Remove HTML tags
    text = re.sub('<[^<]+?>', '', text)
    return text

# Function to calculate sentiment polarity
def calculate_sentiment(text):
    try:
        return TextBlob(text).sentiment.polarity
    except:
        return None

# Parse the XML file
tree = etree.parse('C:\\Users\\aleks\\OneDrive\\Pulpit\\HW3\\Comments.xml')
root = tree.getroot()

# Extract comments data
comments = []
for comment in root.findall('.//row'):
    text = comment.get('Text')  # Assuming Text is the attribute for comment text
    if text:
        comments.append(clean_text(text))

# Create a dataframe
comments_df = pd.DataFrame(comments, columns=['Text'])

# Apply sentiment analysis
comments_df['SentimentPolarity'] = comments_df['Text'].apply(calculate_sentiment)
print(comments_df[['Text', 'SentimentPolarity']].head(20)) # Can be changed depending on how many records we want to display


                                                 Text  SentimentPolarity
0   +1 for `Of course different instructors are go...           0.309524
1   +1 for `This also means listening to your knee...           0.000000
2   +1 for `Knee issues more often than not come f...           0.500000
3   Do you have a video of it being performed or k...           0.000000
4     I have only ever learned blood (artery) chokes.           0.000000
5   If you know the palgwe forms, I believe he's t...           0.200000
6   Agreed in regard to the "traditional" sense......           0.025000
7   @Rophuine that sounds about right, do you have...           0.142857
8   I've heard the same here and there, but I have...           0.050000
9   I've studied in RCJ Machado BJJ which branched...           0.000000
10                      No legitimate ones, anyway ;)           0.250000
11  That article seemed a little light-weight. My ...           0.222500
12  As far as I know, there are only national gun .

The Sentiment Polarity scores range from -1 to 1, where -1 indicates a very negative sentiment, 0 indicates a neutral sentiment, and 1 indicates a very positive sentiment. As seen, many of the results obtained a neutral score. What is interesting, is a scored 1.0000. The greeting "Hi" followed by "Welcome to the site!" is a friendly and warm welcome message, often seen as very positive. On the other hand, the comment with score -0.024107 suggests that the mention of "national gun" could trigger slight negativity due to the neagtive connotations of such word.

### Research Question 2

### *What are the locations of the forum's Users?*

In [15]:
import pandas as pd
import xml.etree.ElementTree as ET

# Parse the XML file
tree = ET.parse('C:\\Users\\aleks\\OneDrive\\Pulpit\\HW3\\Users.xml')
root = tree.getroot()

# Extract user data
users_data = []
for user in root.findall('row'):
    user_data = {
        'UserId': user.attrib.get('Id'),
        'Location': user.attrib.get('Location')  # Assuming the attribute name is 'Location'
    }
    users_data.append(user_data)

# Create a dataframe
df_users = pd.DataFrame(users_data)

# Display the first few rows to check the data
print(df_users.head(100)) # Can be changed depending on how many records we want to display



   UserId               Location
0      -1     on the server farm
1       2          Corvallis, OR
2       3          United States
3       4      Bellevue, WA, USA
4       5                   None
..    ...                    ...
95    103                Glasgow
96    104  Washington, D.C. area
97    105       Hamilton, Canada
98    106          Massachusetts
99    107     Salt Lake City, UT

[100 rows x 2 columns]


In [14]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import geopy.location  # Import geopy.location module

# Assuming df_users is a dataframe created from your XML file with a 'Location' column
geolocator = Nominatim(user_agent="unique_user_agent")  # Replace 'unique_user_agent' with your own user agent

# Adjust the RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=2, max_retries=2, error_wait_seconds=5)

# Geocode the first 10 locations (to avoid excessive requests in this example)
df_users['Location_Coordinates'] = df_users['Location'].head(100).apply(geocode)

# Define functions for extracting latitude and longitude
def extract_latitude(loc):
    if isinstance(loc, geopy.location.Location):
        return loc.latitude
    return None

def extract_longitude(loc):
    if isinstance(loc, geopy.location.Location):
        return loc.longitude
    return None

# Extract latitude and longitude with improved error handling
df_users['Latitude'] = df_users['Location_Coordinates'].apply(extract_latitude)
df_users['Longitude'] = df_users['Location_Coordinates'].apply(extract_longitude)

# Create a map object
m = folium.Map(location=[0, 0], zoom_start=2)

# Add markers to the map for each user location
for i, row in df_users.dropna(subset=['Latitude', 'Longitude']).iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], popup=row['Location']).add_to(m)

# Save the map to an HTML file
m.save('user_locations.html')

Here, we used the Geocoding technique, using Python library *geopy*. By geocoding latitude and longitude coordinates, we can locate the data in the form of city/state/country names. 

After executing the code, the file *user_locations.html* is created. It shows the exact locations for the Users of the Martial Arts Stack Exchange. In oder to display it, do the right click and choose the option "Open with Live Server". The file will show the real-time locations displayed on the world's map. 

### Research Question 3

### *Is there a way to group similar Posts or Users based on certain features?*

In [4]:
import xml.etree.ElementTree as ET
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Parse the XML file
tree = ET.parse('C:\\Users\\aleks\\OneDrive\\Pulpit\\HW3\\Posts.xml')
root = tree.getroot()

# Extracting post data
posts_data = []
for post in root.findall('row'):
    if post.attrib.get('PostTypeId') == '1':  # Assuming 1 indicates a Post
        posts_data.append({
            'Id': post.attrib.get('Id'),
            'Title': post.attrib.get('Title')
        })

# Create dataframe
df_posts = pd.DataFrame(posts_data)

# Check if titles are available
if 'Title' not in df_posts or df_posts['Title'].isnull().all():
    raise ValueError("No post titles available for clustering")

# Using post titles for clustering
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
X = vectorizer.fit_transform(df_posts['Title'])

# Standardize features
scaler = StandardScaler(with_mean=False)
X_scaled = scaler.fit_transform(X)

# Apply K-means clustering
num_clusters = 5  # Adjust the number of clusters based on your data
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X_scaled)

# Add cluster labels to the original dataframe
df_posts['Cluster'] = kmeans.labels_

# Viewing the first few entries with their cluster labels
print(df_posts[['Title', 'Cluster']].head(30)) # Can be changed depending on how many records we want to display

                                                Title  Cluster
0   Best weight lifting exercise to develop a Wing...        0
1   What exercises can I do to strengthen my knees...        2
2   When moving to a new city, what considerations...        2
3   Proper way to take care of boxing gloves and w...        2
4                       How to return to Tae Kwon Do?        2
5               Benefits of an X-Stop in Tae Kwon Do?        2
6   What's the real difference between Ikkyo omote...        2
7   What are the ways to get sweat stains out of a...        2
8      Do all schools of BJJ come from the same root?        2
9                      Doing Miller's One-Step Drills        2
10  What's the best way to sew badges onto a unifo...        2
11  What's the best way to stop a nose bleed quickly?        2
12  How dangerous is it to choke someone unconscio...        2
13            Does ninjutsu qualify as a martial art?        2
14  Is practising techniques on both the left and ...  

c:\Users\aleks\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In order to answer this research question, we decided to implement K-means Clustering. here, we assumed that we want to cluster the Posts data based on their textual content. The procedure of creating such technique is the following:

1. Convert text data (e.g. post titles) into numerical form using TF-IDF vectorization (Text Vectorization).
2. Standardise the features to have a mean of 0 and a standard deviation of 1 (Feature Scaling).
3. Use the K-means algorithm to cluster the Posts.
4. Analyse the clusters in order to understand the common Posts' themes.


To interpret the meaning behind each cluster, we need to seek for common topics or themes present in the Posts' titles, assigned to each cluster. Below, it is the individual interpretation of each of them:

*Cluster 0*
* This cluster seems to focus on training methods or specific techniques within a martial art style.


*Cluster 1*
* This cluster might centre on choosing martial arts schools or dojos - room or hall in which martial arts are practised.

*Cluster 2*
* Titles cover a wide range of topics, including exercise tips, care of equipment, martial arts styles comparison, health and safety in training. 
* The diversity suggests that Cluster 2 might be for posts that don't fit neatly into more narrowly defined themes.

*Cluster 3*
* This cluster might focus on training practices or methodologies. 
